In [1]:
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve2d, fftconvolve
from numpy.fft import rfft2, irfft2

In [3]:
from matplotlib import colors
import matplotlib.animation as animation
from IPython.display import HTML

# Numbers, Arrays and Life

Cellular automata are discrete-space, discrete-time models of spatio-temporal processes. In these models, signal propagation typically involves updating the state of each cell in a grid based on some function of its local neighbors. We will use a toy example, but the techniques you will implement generalize to useful contexts, e.g. spatial statistics or image processing.

We will use John Conway's [Game of Life](https://en.wikipedia.org/wiki/Conway's_Game_of_Life) to practice our manipulation of numerical arrays. In particular, we will see how to use the vectorization, views, convolution and Fourier transform capabilities of `numpy` to solve the same problem in different ways.

See Wikipedia for detailed rules. Here is a summary:

```
Any live cell with fewer than two live neighbors dies, as if caused by under-population.
Any live cell with two or three live neighbors lives on to the next generation.
Any live cell with more than three live neighbors dies, as if by over-population.
Any dead cell with exactly three live neighbors becomes a live cell, as if by reproduction.
```

There are two common ways to count neighbors for cells that are at the edge of the grid. With periodic boundary conditions, the grid wraps around (so it is a torus), that is, the coordinate -1 is mapped to the max_coord, and max_coord + 1 is mapped to 0. Alternatively, just consider any neighbor outside the grid to be equal to zero. For simplicity, we have chosen a life configuration where both strategies give the same results, so you can implement either strategy.

### Initial grid configuration

In [4]:
gun = np.load('gosper_gun.npy')
init_grid = np.zeros((64, 64)).astype('int')
for y, x in gun:
    init_grid[8+x, 8+y] = 1

### Utility function to show animation

In [5]:
def animate(i):
    """Function to display animation for iteration i."""
    global grid
    grid = step(grid)
    im.set_array(grid)
    return im,

**1**. Write a function to play one step in the Game of Life. The function must be named `step` and take a single argument `grid` which is a `numpy` array containing the initial configuration, and return a new `numpy` array containing the updated configuration. Use for loops to implement this first version of the step function.

In [6]:
## Your solution here 





In [7]:
def step(grid):
    """One iteration of game of life.
    Returns updated grid."""

    new_grid = grid.copy()
    
    ncols, nrows = grid.shape
    for c in range(ncols):
        for r in range(nrows):
            nbr_sum = 0
            for i in range(c-1, c+2):
                x = i % ncols
                for j in range(r-1, r+2):
                    y = j % nrows
                    nbr_sum += grid[x, y]
            nbr_sum -= grid[c, r]
            if (grid[c, r] == 0) and (nbr_sum == 3):
                new_grid[c, r] = 1
            elif (grid[c, r] == 1):
                if ((nbr_sum < 2) or (nbr_sum > 3)):
                    new_grid[c, r] = 0
    return new_grid

In [8]:
fig = plt.figure(figsize=(5, 5))
grid = init_grid.copy()
im = plt.imshow(grid, animated=True, interpolation='nearest', cmap='gray')
plt.close(fig)

anim = animation.FuncAnimation(fig, animate, 
                               frames=60, interval=50, blit=True)
HTML(anim.to_html5_video())

**2**. Rewrite the step function using vectorization. That is, use eight different views of the grid to calculate the neighbor sum instead of double for loops.

In [9]:
# Your solution here





In [10]:
def step(grid):
    """One iteration of game of life.
    Returns updated grid."""

    nbr_sum = (grid[0:-2,0:-2] + grid[0:-2,1:-1] + grid[0:-2,2:] +
               grid[1:-1,0:-2]                   + grid[1:-1,2:] +
               grid[2:,0:-2]   + grid[2:,1:-1]   + grid[2:,2:])
    
    birth = (nbr_sum == 3) & (grid[1:-1,1:-1] == 0)
    survive = ((nbr_sum == 2) | (nbr_sum == 3)) & (grid[1:-1,1:-1]==1)
    
    new_grid = np.zeros_like(grid)
    new_grid[1:-1,1:-1][birth|survive]  = 1
    new_grid[1:-1,1:-1][~(birth|survive)] = 0
    return new_grid

In [11]:
fig = plt.figure(figsize=(5, 5))
grid = init_grid.copy()
im = plt.imshow(grid, animated=True, interpolation='nearest', cmap='gray')
plt.close(fig)

anim = animation.FuncAnimation(fig, animate, 
                               frames=60, interval=50, blit=True)
HTML(anim.to_html5_video())

**3a**. A discrete 2D convolution generates a weighted sum of a 2D grid, with the weights given by a 2D kernel. For example, the kernel

```
0 1 0
1 1 1
0 1 0
```

would result in summing the current location and the N, E, S, W neighbors.

Use the `convolve2d` function from `scipy.signal` (already imported for you in this notebook) to implement the 3rd version of the `step` function with a suitable kernel and with `mode=same` to preserve the grid size across iterations.

In [12]:
# Your solution here





In [13]:
kernel = np.ones((3,3))
kernel[1,1] = 0

def step(grid):
    """One iteration of game of life.
    Returns updated grid."""
    nbr = convolve2d(grid, kernel, mode='same', boundary='wrap')
    new = np.ones_like(grid)
    new = np.where((grid == 0) & (nbr == 3), 1, grid)
    new[(grid == 1) & ((nbr <= 1) | (nbr > 3))] = 0
    return new

In [14]:
fig = plt.figure(figsize=(5, 5))
grid = init_grid.copy()
im = plt.imshow(grid, animated=True, interpolation='nearest', cmap='gray')
plt.close(fig)

anim = animation.FuncAnimation(fig, animate, 
                               frames=60, interval=50, blit=True)
HTML(anim.to_html5_video())

**3b**. One way to multiply two numbers that you are familiar with is to move to the log domain where addition is equivalent to multiplication. For example, instead of calculating $100 \times 1000$, we can calculate $\exp(\log(100) + \log(1000))$ to get the same result to roundoff error. In the same way, a convolution is equivalent to multiplication in the Fourier domain.  

Implement the fourth version of the `step` function using the `fftconvolve` function for real Fast Fourier Transforms from the `scipy.signal` package. Because of rounding errors, you need to round the results returned by `fftconvolve`.

In [15]:
# Your solution here






In [16]:
kernel = np.ones((3,3))
kernel[1,1] = 0

def step(grid):
    """One iteration of game of life.
    Returns updated grid."""  
    nbr = np.around(fftconvolve(grid, kernel, mode='same'))
    new = np.ones_like(grid)
    new = np.where((grid == 0) & (nbr == 3), 1, grid)
    new[(grid == 1) & ((nbr <= 1) | (nbr > 3))] = 0
    return new

In [17]:
fig = plt.figure(figsize=(5, 5))
grid = init_grid.copy()
im = plt.imshow(grid, animated=True, interpolation='nearest', cmap='gray')
plt.close(fig)

anim = animation.FuncAnimation(fig, animate, 
                               frames=60, interval=50, blit=True)
HTML(anim.to_html5_video())

In [18]:
np.random.uniform(0, 1, grid.shape)

array([[ 0.75679944,  0.96449772,  0.90441775, ...,  0.44378814,
         0.25345354,  0.47181773],
       [ 0.51900626,  0.81039199,  0.43694671, ...,  0.31775469,
         0.09612661,  0.93072276],
       [ 0.42353589,  0.34070907,  0.76178319, ...,  0.31185637,
         0.11642768,  0.62228758],
       ..., 
       [ 0.62999777,  0.82110852,  0.0088721 , ...,  0.72288221,
         0.56393521,  0.52346735],
       [ 0.33505715,  0.15552663,  0.89790542, ...,  0.95558807,
         0.70971818,  0.15879394],
       [ 0.7490564 ,  0.52985953,  0.27873871, ...,  0.72500064,
         0.38238423,  0.54778653]])

**4**. Modify 3a to model the [Forest Fire model](https://en.wikipedia.org/wiki/Forest-fire_model) instead of the Game of Life, with `f=0.001` and `p=0.1`.

In [19]:
# Your solution here






In [20]:
kernel = np.ones((3,3))
kernel[1,1] = 0

def step(grid, p=0.1, f=0.001):
    """One iteration of game of life.
    Returns updated grid."""
    burn = grid == 2
    
    nbr = convolve2d(burn, kernel, mode='same', boundary='wrap')
    
    # A burning cell turns into an empty cell
    grid = np.where(grid == 2, 0, grid)
    
    # A tree will burn if at least one neighbor is burning
    grid = np.where((grid == 1) & (nbr >= 1), 2, grid)
    
    #A tree ignites with probability f even if no neighbor is burning
    cond = np.random.uniform(0, 1, grid.shape) < f
    grid = np.where((grid == 1) & (nbr == 0) & cond, 2, grid)

    # An empty space fills with a tree with probability p
    cond = np.random.uniform(0, 1, grid.shape) < p
    grid = np.where((grid == 0) & cond, 1, grid)
    
    return grid

In [21]:
cmap = colors.ListedColormap(['black', 'green', 'red'])

fig = plt.figure(figsize=(5, 5))
grid = init_grid.copy()
im = plt.imshow(grid, animated=True, interpolation='nearest', cmap=cmap, vmax=2)
plt.close(fig)

anim = animation.FuncAnimation(fig, animate, 
                               frames=100, interval=50, blit=True)
HTML(anim.to_html5_video())

In [3]:
import numpy as np

np.random.rand?